<a href="https://colab.research.google.com/github/zhengwu123/MLKit_practice/blob/master/age%2Cgender%2Cethnicity%20prediciton%20using%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#upload our data to colab
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [17]:
#unzip file
!unzip archive.zip

Archive:  archive.zip
  inflating: age_gender.csv          


In [18]:

import numpy as np 
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D
#read file from agen_gender.csv
data = pd.read_csv('./age_gender.csv')

# Converting pixels into numpy array
data['pixels']=data['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))

data.head()

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,"[129.0, 128.0, 128.0, 126.0, 127.0, 130.0, 133..."
1,1,2,0,20161219222752047.jpg.chip.jpg,"[164.0, 74.0, 111.0, 168.0, 169.0, 171.0, 175...."
2,1,2,0,20161219222832191.jpg.chip.jpg,"[67.0, 70.0, 71.0, 70.0, 69.0, 67.0, 70.0, 79...."
3,1,2,0,20161220144911423.jpg.chip.jpg,"[193.0, 197.0, 198.0, 200.0, 199.0, 200.0, 202..."
4,1,2,0,20161220144914327.jpg.chip.jpg,"[202.0, 205.0, 209.0, 210.0, 209.0, 209.0, 210..."


In [19]:
# augment data

X = np.array(data['pixels'].tolist())

## Converting pixels from 1D to 3D
X = X.reshape(X.shape[0],48,48,1)

**part 1. Use google keras cnn to predict gender.**

In [51]:
#gender task.

#split data

y = data['gender']

X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(
    X, y, test_size=0.20
)

In [53]:
#build our CNN model
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Dense, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Model,Sequential
model_gender = Sequential()

#add conv1
model_gender.add(Conv2D(32,(3,3),activation='relu',strides=1,padding="VALID", input_shape=(48,48,1)))
#add pool1
model_gender.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#add conv2
model_gender.add(Conv2D(64, (3, 3),strides=(3,3),activation='relu',padding="VALID"))
#add pool2
model_gender.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_gender.add(Flatten())
model_gender.add(Dense(512, activation='relu'))
model_gender.add(Dense(1, activation='sigmoid'))
model_gender.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
#start training our model
model_gender.summary()
#fit our model
r_gender = model_gender.fit( X_train_gender, y_train_gender, validation_split=0.1,batch_size = 64, epochs=15)




Epoch 1/15
267/267 [==============================] - 21s 80ms/step - loss: 0.7873 - accuracy: 0.7368 - val_loss: 0.3947 - val_accuracy: 0.8271
Epoch 2/15
267/267 [==============================] - 21s 80ms/step - loss: 0.3770 - accuracy: 0.8289 - val_loss: 0.3534 - val_accuracy: 0.8413
Epoch 3/15
267/267 [==============================] - 21s 80ms/step - loss: 0.3395 - accuracy: 0.8483 - val_loss: 0.3325 - val_accuracy: 0.8556
Epoch 4/15
267/267 [==============================] - 21s 80ms/step - loss: 0.3101 - accuracy: 0.8639 - val_loss: 0.3332 - val_accuracy: 0.8556
Epoch 5/15
267/267 [==============================] - 21s 80ms/step - loss: 0.2850 - accuracy: 0.8774 - val_loss: 0.3482 - val_accuracy: 0.8571
Epoch 6/15
267/267 [==============================] - 21s 80ms/step - loss: 0.2645 - accuracy: 0.8857 - val_loss: 0.3465 - val_accuracy: 0.8529
Epoch 7/15
267/267 [==============================] - 21s 80ms/step - loss: 0.2433 - accuracy: 0.8984 - val_loss: 0.3382 - val_accuracy:

In [54]:
#print test lost and accuracy
loss, acc = model_gender.evaluate(X_test,y_test,verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

Test loss: 0.22098411619663239
Test Accuracy: 0.934191107749939


In [68]:
#predict ethnicity
y_ethnicity = data['ethnicity']

X_train_ethnicity, X_test_ethnicity, y_train_ethnicity, y_test_ethnicity = train_test_split(
    X, y_ethnicity, test_size=0.2)

In [57]:
#print y_label
print(y_ethnicity)

0        2
1        2
2        2
3        2
4        2
        ..
23700    0
23701    1
23702    2
23703    2
23704    0
Name: ethnicity, Length: 23705, dtype: int64


**part 2: predict ethnicity**


In [75]:
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Dense, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Model,Sequential

model_ethnicity = tf.keras.Sequential([
    L.InputLayer(input_shape=(48,48,1)),
    L.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    L.MaxPooling2D((2, 2)),
    L.Conv2D(64, (3, 3), activation='relu'),
    L.MaxPooling2D((2, 2)),
    L.Flatten(),
    L.Dense(64, activation='relu'),
    L.Dropout(rate=0.5),
    L.Dense(5)
])
model_ethnicity.compile(optimizer='rmsprop',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model_ethnicity.summary()
#fit our model
r_ethnicity = model_ethnicity.fit( X_train_ethnicity, y_train_ethnicity, validation_split=0.1,batch_size = 64, epochs=10)

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_67 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_35 (Flatten)         (None, 6400)              0         
_________________________________________________________________
dense_70 (Dense)             (None, 64)                409664    
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)              

In [104]:
#print test lost and accuracy
loss, acc = model_ethnicity.evaluate(X_test_ethnicity,y_test_ethnicity,verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

Test loss: 0.8049358129501343
Test Accuracy: 0.7342332601547241


**part 3: predict age.**


In [94]:
# split data
y_age = data['age']

X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(
    X, y_age, test_size=0.20)

In [87]:
#print age label
print(y_age)

0         1
1         1
2         1
3         1
4         1
         ..
23700    99
23701    99
23702    99
23703    99
23704    99
Name: age, Length: 23705, dtype: int64


In [103]:
from tensorflow.keras.layers import BatchNormalization
y = data['age'].values

model = tf.keras.Sequential([
    L.InputLayer(input_shape=(48,48,1)),
    L.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    L.BatchNormalization(),
    L.MaxPooling2D((2, 2)),
    L.Conv2D(64, (3, 3), activation='relu'),
    L.MaxPooling2D((2, 2)),
    L.Conv2D(128, (3, 3), activation='relu'),
    L.MaxPooling2D((2, 2)),
    L.Flatten(),
    L.Dense(64, activation='relu'),
    L.Dropout(rate=0.5),
    L.Dense(1, activation='relu')
])

sgd = tf.keras.optimizers.SGD(momentum=0.9)

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])
model.summary()
history = history = model.fit(
    X_train_age, y_train_age, epochs=20, validation_split=0.1, batch_size=64)

Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_100 (Conv2D)          (None, 46, 46, 32)        320       
_________________________________________________________________
batch_normalization_35 (Batc (None, 46, 46, 32)        128       
_________________________________________________________________
max_pooling2d_108 (MaxPoolin (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_109 (MaxPoolin (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_110 (MaxPoolin (None, 4, 4, 128)       

KeyboardInterrupt: ignored

In [31]:
#print performance of our model
mse, mae = model_age.evaluate(X_test,y_test,verbose=0)
print('Test Mean squared error: {}'.format(mse))
print('Test Mean absolute error: {}'.format(mae))

Test Mean squared error: -1406356976304128.0
Test Mean absolute error: 0.04809112101793289
